In [50]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/marketing_campaign_data_messy.csv')
df

,Campaign_ID,Campaign_Name,Start_Date,End_Date,Channel,Impressions,Clicks,Spend,Conversions,Active,Clicks,Campaign_Tag
0,CMP-00001,Q4_Summer_CMP-00001,2023-11-24 00:00:00,2023-12-13,TikTok,16795,197,$102.82,20.0,Y,NaN,TI
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06 00:00:00,2023-05-12,Facebook,1860,30,24.33,1.0,0,NaN,FA
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13 00:00:00,2023-12-20,Email,77820,843,1323.39,51.0,No,NaN,EM
3,CMP-00004,Q1_BlackFriday_CMP-00004,2023-10-30,2023-11-03,TikTok,55886,2019,2180.38,135.0,True,NaN,TI
4,CMP-00005,Q2_Winter_CMP-00005,2023-04-22 00:00:00,2023-04-23,Facebook,7265,169,252.44,30.0,Yes,NaN,FA
...,...,...,...,...,...,...,...,...,...,...,...,...
2015,CMP-00400,Q3_Summer_CMP-00400,2023-10-31 00:00:00,2023-11-13,TikTok,30592,586,$503.95,77.0,1,NaN,TI
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01 00:00:00,2023-09-26,Google Ads,20097,897,1641.0,162.0,0,NaN,GO
2017,CMP-01050,Q2_Launch_CMP-01050,2023-02-09 00:00:00,2023-02-21,Instagram,33254,1117,883.82,214.0,0,NaN,IN
2018,CMP-01118,Q4_Winter_CMP-01118,2023-03-30 00:00:00,2023-04-27,Facebook,68728,2960,4198.5,591.0,Yes,NaN,FA


In [51]:
#HEADER CLEANING
#cleaning spaces between names in "" also to lower case and also replace empty space with underscore
print(df.columns.to_list())

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print ('FIX APPLIED')
print(df.columns.to_list())

[' Campaign_ID ', 'Campaign_Name', 'Start_Date', 'End_Date', 'Channel', 'Impressions', 'Clicks ', 'Spend', 'Conversions', 'Active', 'Clicks', 'Campaign_Tag']
FIX APPLIED
['campaign_id', 'campaign_name', 'start_date', 'end_date', 'channel', 'impressions', 'clicks', 'spend', 'conversions', 'active', 'clicks', 'campaign_tag']


In [52]:
#TYPE CONVERSION AND CURRENCY
dirty_spend_mask = df['spend'].str.contains('\$')
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(10))

df['spend'] = df['spend'].astype(str).str.replace(r'[^\d.-]', '', regex = True)
df['spend'] = pd.to_numeric(df['spend'], errors = 'coerce')

print('FIX APPLIED')
print(df.loc[dirty_spend_mask,['campaign_id','spend']].head(10))

   campaign_id     spend
0    CMP-00001   $102.82
21   CMP-00022   $2428.4
22   CMP-00023  $4726.22
31   CMP-00032  $2759.35
32   CMP-00033  $2393.02
36   CMP-00037  $2136.63
37   CMP-00038  $1118.15
40   CMP-00041   $205.48
53   CMP-00054   $919.99
55   CMP-00056  $3219.35
FIX APPLIED
   campaign_id    spend
0    CMP-00001   102.82
21   CMP-00022  2428.40
22   CMP-00023  4726.22
31   CMP-00032  2759.35
32   CMP-00033  2393.02
36   CMP-00037  2136.63
37   CMP-00038  1118.15
40   CMP-00041   205.48
53   CMP-00054   919.99
55   CMP-00056  3219.35


<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-540916591.py:2: SyntaxWarning: invalid escape sequence '\$'
  dirty_spend_mask = df['spend'].str.contains('\$')


In [53]:
#Categorical Typos; different values returning the same meaning. Making it one
print(df['channel'].unique())

cleanup_mask = {
    'Facebok':'Facebook',
    'Insta_gram':'Instagram',
    'Tik_Tok': 'TikTok',
    'Gogle': 'Google Ads',
    'E-mail': 'Email',
    'N/a': np.nan #Handles ghost values
}
df['channel'] = df['channel'].replace(cleanup_mask)

print('FIX APPLIED')
print(df['channel'].unique())

['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' 'E-mail' nan 'Gogle'
 'Tik_Tok' 'Facebok' 'Insta_gram']
FIX APPLIED
['TikTok' 'Facebook' 'Email' 'Instagram' 'Google Ads' nan]


In [54]:
#Handling mixed booleans yes and no, true of false
print(df['active'].unique())

bool_map = {
    'yes': True,
    'Y': True,
    '0': False,
    '1': True,
    'no': False,
    'true': True,
    'false': False
}
df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)

print('FIX APPLIED')
print(df['active'].unique())

['Y' '0' 'No' 'True' 'Yes' '1' 'False']
FIX APPLIED
[ True False]


/tmp/ipython-input-1216061899.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['active'] = df['active'].map(bool_map).fillna(False).astype(bool)


In [55]:
#DATE PARSING
print(df['start_date'].dtype)

df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')

print('FIX APPLIED')
print(df['start_date'].dtype)

object
FIX APPLIED
datetime64[ns]


In [56]:
#REMOVE DUPLICATE COLUMNS
df = df.loc[:,~df.columns.duplicated()]

In [57]:
#LOGICAL INTEGRITY (Clicks vs Impression)
impossible_mask = df['clicks'] > df['impressions']
print(df.loc[impossible_mask,['campaign_id','clicks','impressions']].head(10))

Empty DataFrame
Columns: [campaign_id, clicks, impressions]
Index: []


In [58]:
#LOGICAL INTEGRITY (TIME TRAVEL) chronological order between start date and end date
time_travel_mask = df['start_date'] > df['end_date']
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(10))

df.loc[time_travel_mask, 'end_date'] = df.loc[time_travel_mask, 'start_date'] + pd.Timedelta(days=30)

print('FIX APPLIED')
print(df.loc[time_travel_mask,['campaign_id','start_date','end_date']].head(10))

    campaign_id start_date   end_date
23    CMP-00024 2023-05-06 2023-05-01
54    CMP-00055 2023-09-01 2023-08-27
71    CMP-00072 2023-02-01 2023-01-27
156   CMP-00157 2023-12-06 2023-12-01
200   CMP-00201 2023-01-11 2023-01-06
218   CMP-00219 2023-10-03 2023-09-28
221   CMP-00222 2023-08-26 2023-08-21
280   CMP-00281 2023-12-10 2023-12-05
303   CMP-00304 2023-05-17 2023-05-12
346   CMP-00347 2023-11-07 2023-11-02
FIX APPLIED
    campaign_id start_date   end_date
23    CMP-00024 2023-05-06 2023-06-05
54    CMP-00055 2023-09-01 2023-10-01
71    CMP-00072 2023-02-01 2023-03-03
156   CMP-00157 2023-12-06 2024-01-05
200   CMP-00201 2023-01-11 2023-02-10
218   CMP-00219 2023-10-03 2023-11-02
221   CMP-00222 2023-08-26 2023-09-25
280   CMP-00281 2023-12-10 2024-01-09
303   CMP-00304 2023-05-17 2023-06-16
346   CMP-00347 2023-11-07 2023-12-07


In [59]:
#HANDLING OUTLIERS
#Check for massive outliers in column spend
Q1 = df['spend'].quantile(0.25)
Q3 = df['spend'].quantile(0.75)

IQR = Q3 - Q1
upper_limit = Q3 + (3 * IQR) #maximum value in the column spent

outlier_mask = df['spend'] > upper_limit
print(df.loc[outlier_mask,['campaign_id','spend']].head(10))

print('FIX APPLIED')
df.loc[outlier_mask, 'spend'] = upper_limit
print(df.loc[outlier_mask,['campaign_id','spend']].head(10))

     campaign_id      spend
789    CMP-00790  500000.00
1443   CMP-01444    8921.51
1460   CMP-01461  500000.00
1718   CMP-01719  500000.00
1754   CMP-01755  500000.00
1781   CMP-01782  500000.00
FIX APPLIED
     campaign_id      spend
789    CMP-00790  8603.5375
1443   CMP-01444  8603.5375
1460   CMP-01461  8603.5375
1718   CMP-01719  8603.5375
1754   CMP-01755  8603.5375
1781   CMP-01782  8603.5375


In [61]:
#FEATURE EXTRACTION
print(df['campaign_name'].head(10))

df['season'] = df['campaign_name'].str.extract(r'Q\d_([^_]+)_')
print('FIX APPLIED')
print(df[['campaign_name', 'season']].head(10))

0         Q4_Summer_CMP-00001
1         Q1_Launch_CMP-00002
2         Q3_Winter_CMP-00003
3    Q1_BlackFriday_CMP-00004
4         Q2_Winter_CMP-00005
5    Q4_BlackFriday_CMP-00006
6         Q3_Launch_CMP-00007
7         Q4_Launch_CMP-00008
8    Q4_BlackFriday_CMP-00009
9         Q2_Winter_CMP-00010
Name: campaign_name, dtype: object
FIX APPLIED
              campaign_name       season
0       Q4_Summer_CMP-00001       Summer
1       Q1_Launch_CMP-00002       Launch
2       Q3_Winter_CMP-00003       Winter
3  Q1_BlackFriday_CMP-00004  BlackFriday
4       Q2_Winter_CMP-00005       Winter
5  Q4_BlackFriday_CMP-00006  BlackFriday
6       Q3_Launch_CMP-00007       Launch
7       Q4_Launch_CMP-00008       Launch
8  Q4_BlackFriday_CMP-00009  BlackFriday
9       Q2_Winter_CMP-00010       Winter
